In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [5]:
import os


root_dir = os.path.abspath('../')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [ ]:
# def build_data():
    
#     import re
#     import json
    
#     p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
#     seg_file = os.path.join(data_path, 'final_baidu-23w.txt')
    
#     delimiter='△△△'
    
#     replace_chars = [
#         '\x97',
#         '\uf076',
#         "\ue405",
#         "\ue105",
#         "\ue415",
#         '\x07',
#         '\x7f',
#         '\u3000',
#         '\xa0',
#         ' '
#     ]
#     with open(seg_file) as fin, open(train_path, 'w') as train_f, open(valid_path, 'w') as valid_f:
#         train_f.write(f'0{delimiter}1\n')
#         valid_f.write(f'0{delimiter}1\n')
#         for line in fin:
#             line = line.strip()
#             if not line:
#                 continue
                
#             import random
#             score = random.random()
            
#             fout = train_f if score > 0.3 else valid_f
#             words = []
#             flags = []
#             for word, flag in p.findall(line):
#                 char_list = [c if c not in replace_chars  else 'unk' for c in list(word)]
                
#                 tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 1)
                
#                 if len(char_list) != len(tag_list):
#                     print(line)
#                     print(word, flag)
#                     print(char_list, tag_list)
                    
#                 words.extend(char_list)
#                 flags.extend(tag_list)
                
#             assert len(words) == len(flags)
                
#             fout.write(delimiter.join([
#                 ' '.join(flags),
#                 ' '.join(words)
#             ]))
#             fout.write('\n')
            
            
            
# build_data()
    

In [ ]:
# 正常训练

from modules import BertNerData as NerData

data = NerData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    is_cls=False,
    max_seq_len=64,
    batch_size=32
    
)

import torch
import torch.nn as nn
from importlib import reload
from modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from modules.train import train
reload(train)
num_epochs = 30
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [8]:
# 恢复训练

from blnlp import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

learner = tagger.learner
num_epochs = 5
learner.load_model()
learner.fit(num_epochs, target_metric='f1')


DEBUG:web.utils.common:get_data cost 102.320187s


DEBUG:web.utils.common:get_data cost 51.401947s


INFO:root:Resuming train... Current epoch 0.


In [19]:
type(learner.data.train_dl)

modules.data.bert_data.DataLoaderForTrain

In [ ]:
learner.save_model()

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [5]:
from blnlp import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


In [6]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

DEBUG:web.utils.common:get_data cost 0.027648s


DEBUG:web.utils.common:text_array_for_predict cost 0.140504s



0.14156174659729004


[[('美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司', 'nt'),
  ('始创', 'v'),
  ('于', 'p'),
  ('2004年', 't'),
  (',', 'w'),
  ('是', 'v'),
  ('中国', 'ns'),
  ('健康', 'an'),
  ('体检', 'vn'),
  ('和', 'c'),
  ('医疗', 'n'),
  ('服务', 'vn'),
  ('集团', 'n'),
  (',', 'w'),
  ('总部', 'n'),
  ('位于', 'v'),
  ('上海', 'ns'),
  (',', 'w'),
  ('深耕布局', 'v'),
  ('北京', 'ns'),
  ('、', 'w'),
  ('深圳', 'ns'),
  ('、', 'w'),
  ('沈阳', 'ns'),
  ('、', 'w'),
  ('广州', 'ns'),
  ('、', 'w'),
  ('成都', 'ns'),
  ('、', 'w'),
  ('武汉', 'ns'),
  ('、...', 'w')]]

In [ ]:
tagger.learner.data.is_meta